In [1]:
import os
import re

import numpy as np
import pandas as pd

# os.chdir("/home/yujia/Project/2023-07-20-individual_MR/dat/02_base_data/")

In [2]:
CAD = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/download/26343387-GCST003116-EFO_0000378-build37.f.tsv.gz", sep="\t", compression = 'gzip')

print("Total SNP in the GWAS data: ")
print(CAD.shape)

Total SNP in the GWAS data: 
(9455778, 13)


In [3]:
CAD

,variant_id,chromosome,base_pair_location,effect_allele,other_allele,effect_allele_frequency,beta,standard_error,p_value,nstudy,odds_ratio,ci_lower,ci_upper
0,rs143225517,1,751756,C,T,0.158264,0.013006,0.017324,0.452802,35,NaN,NaN,NaN
1,rs3094315,1,752566,A,G,0.763018,-0.005243,0.015765,0.739460,36,NaN,NaN,NaN
2,rs3131972,1,752721,G,A,0.740969,-0.003032,0.015638,0.846265,36,NaN,NaN,NaN
3,rs3131971,1,752894,C,T,0.744287,0.004640,0.016238,0.775066,37,NaN,NaN,NaN
4,rs61770173,1,753405,A,C,0.775368,-0.006291,0.016708,0.706526,36,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9455773,rs5771014,22,51216731,C,T,0.252603,-0.016457,0.039497,0.676924,30,NaN,NaN,NaN
9455774,rs147512294,22,51218377,C,G,0.070224,0.037570,0.028974,0.194746,29,NaN,NaN,NaN
9455775,rs28729663,22,51219006,A,G,0.172818,0.010715,0.018136,0.554656,31,NaN,NaN,NaN
9455776,rs114553188,22,51222100,T,G,0.051709,-0.047162,0.029173,0.105959,33,NaN,NaN,NaN


In [4]:
# rename the data
CAD.rename(columns={"chromosome":"CHR", "base_pair_location":"BP", "variant_id":"SNP", "effect_allele":"A1", "other_allele":"A2", "effect_allele_frequency":"EAF", "beta":"BETA", "standard_error":"SE", "p_value":"P"}, inplace=True)

# convert EAF to MAF
CAD['MAF'] = CAD['EAF'].apply(lambda x : x if x <= 0.5 else 1-x)

# Uppercase the A1 and A2
CAD['A1'] = CAD['A1'].str.upper()
CAD['A2'] = CAD['A2'].str.upper()

In [5]:
CAD

,SNP,CHR,BP,A1,A2,EAF,BETA,SE,P,nstudy,odds_ratio,ci_lower,ci_upper,MAF
0,rs143225517,1,751756,C,T,0.158264,0.013006,0.017324,0.452802,35,NaN,NaN,NaN,0.158264
1,rs3094315,1,752566,A,G,0.763018,-0.005243,0.015765,0.739460,36,NaN,NaN,NaN,0.236982
2,rs3131972,1,752721,G,A,0.740969,-0.003032,0.015638,0.846265,36,NaN,NaN,NaN,0.259031
3,rs3131971,1,752894,C,T,0.744287,0.004640,0.016238,0.775066,37,NaN,NaN,NaN,0.255713
4,rs61770173,1,753405,A,C,0.775368,-0.006291,0.016708,0.706526,36,NaN,NaN,NaN,0.224632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9455773,rs5771014,22,51216731,C,T,0.252603,-0.016457,0.039497,0.676924,30,NaN,NaN,NaN,0.252603
9455774,rs147512294,22,51218377,C,G,0.070224,0.037570,0.028974,0.194746,29,NaN,NaN,NaN,0.070224
9455775,rs28729663,22,51219006,A,G,0.172818,0.010715,0.018136,0.554656,31,NaN,NaN,NaN,0.172818
9455776,rs114553188,22,51222100,T,G,0.051709,-0.047162,0.029173,0.105959,33,NaN,NaN,NaN,0.051709


In [6]:
# reorder the column
cols = ["CHR", "BP", "SNP", "A1", "A2", "SE", "P", "BETA", "MAF", "EAF"]
CAD = CAD[cols]

# Remove NaN row
CAD = CAD.dropna()

# convert to correct column type
CAD = CAD.astype({'CHR':'int', 'BP':'int', "SNP":'str', 'A1':'str', 'A2':'str', 'SE':'float', 'P':'float', 'BETA':'float', 'MAF':'float', 'EAF':'float'})

# remove SNP without SNPID
CAD = CAD[CAD['SNP'].str.startswith('rs')]

# save the formatted results
CAD.to_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC/CAD_formated_nadrop.tsv.gz", sep="\t", index=False)

print("SNP left after basic QC: ")
print(CAD.shape)

SNP left after basic QC: 
(8622850, 10)


In [7]:
CAD

,CHR,BP,SNP,A1,A2,SE,P,BETA,MAF,EAF
0,1,751756,rs143225517,C,T,0.017324,0.452802,0.013006,0.158264,0.158264
1,1,752566,rs3094315,A,G,0.015765,0.739460,-0.005243,0.236982,0.763018
2,1,752721,rs3131972,G,A,0.015638,0.846265,-0.003032,0.259031,0.740969
3,1,752894,rs3131971,C,T,0.016238,0.775066,0.004640,0.255713,0.744287
4,1,753405,rs61770173,A,C,0.016708,0.706526,-0.006291,0.224632,0.775368
...,...,...,...,...,...,...,...,...,...,...
9455773,22,51216731,rs5771014,C,T,0.039497,0.676924,-0.016457,0.252603,0.252603
9455774,22,51218377,rs147512294,C,G,0.028974,0.194746,0.037570,0.070224,0.070224
9455775,22,51219006,rs28729663,A,G,0.018136,0.554656,0.010715,0.172818,0.172818
9455776,22,51222100,rs114553188,T,G,0.029173,0.105959,-0.047162,0.051709,0.051709


In [11]:
# =============================
# Step 2: remove SNPs with MAF < 0.01
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC/CAD_formated_nadrop.tsv.gz |" + \
      "awk 'NR==1 || ($9 > 0.01) {print}' |" + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC/CAD.standardGWASQC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC/CAD.standardGWASQC.gz | wc -l"
print("SNP left after removing SNPs with MAF < 0.01: ")
print(os.popen(cmd).read())

SNP left after removing SNPs with MAF < 0.01: 
8390372



In [12]:
# =============================
# Step 3: remove duplicate SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC/CAD.standardGWASQC.gz |" + \
      "awk '{seen[$3]++; if(seen[$3]==1){ print}}' |" + \
      "gzip - > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC/CAD.nodup.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC/CAD.nodup.gz | wc -l"
print("SNP left after removing duplicate SNPs: ")
print(os.popen(cmd).read())

SNP left after removing duplicate SNPs: 
8390372



In [13]:
# =============================
# Step 4: keep non-ambiguous SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC/CAD.nodup.gz | " + \
      "awk '!( ($4==\"A\" && $5==\"T\") || ($4==\"T\" && $5==\"A\") || ($4==\"G\" && $5==\"C\") || ($4==\"C\" && $5==\"G\")) {print}' | " + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC/CAD.QC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/CAD/QC/CAD.QC.gz | wc -l"
print("SNP left after removing ambiguous SNPs: ")
print(os.popen(cmd).read())

SNP left after removing ambiguous SNPs: 
7092561

